# Advanced document indexing

# Splitting and ingesting HTML content

## Splitting and ingesting the content of a single URL (on Cornwall)

### Preparing the Chroma DB collections

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import getpass

OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY')

Enter your OPENAI_API_KEY ········


In [2]:
corwnall_granular_collection = Chroma( #A
    collection_name="cornwall_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_granular_collection.reset_collection() #B
# A Create a Chorma DB collection
# B Reset the collection in case it already exists 

In [3]:
corwnall_coarse_collection = Chroma( #A 
    collection_name="cornwall_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_coarse_collection.reset_collection() #B
# A Create a Chorma DB collection
# B Reset the collection in case it already exists 

### Loading the HTML content with the AsyncHtmlLoader 

In [4]:
from langchain_community.document_loaders import AsyncHtmlLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
destination_url = "https://en.wikivoyage.org/wiki/Cornwall"

In [6]:
html_loader = AsyncHtmlLoader(destination_url)

In [7]:
docs = html_loader.load()

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.19it/s]


In [8]:
len(docs)

1

### Splitting into granular chunks with the HTMLSectionSplitter 

In [9]:
from langchain_text_splitters import HTMLSectionSplitter

In [10]:
headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]
html_section_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

In [11]:
def split_docs_into_granular_chunks(docs):
    all_chunks = []
    for doc in docs:
        html_string = doc.page_content #A
        temp_chunks = html_section_splitter.split_text(html_string) #B
        all_chunks.extend(temp_chunks) 

    return all_chunks

#A Extract the HTML text from the document
#B Each chunk is a H1 or H2 HTML section

In [12]:
granular_chunks = split_docs_into_granular_chunks(docs)

#### Ingesting granular chunks

In [13]:
corwnall_granular_collection.add_documents(documents=granular_chunks)

['b9c14bb0-df2b-419b-b9a1-fbfd86baa2f8',
 'e4abd12a-76bc-43b8-8193-a8edaef6c3b7',
 '2a474d9d-126b-41db-9ca5-c4c40e867042',
 '64fd22d1-940d-4aef-b669-1c4f2be252a8',
 'e3a98100-6ccc-410c-aa0d-43bf771f4786',
 '9ca7f437-779f-46aa-aca1-20f2d27662c6',
 '6e4f6c70-326e-4ecb-81a2-f2f461546885',
 '58311f9b-5272-499b-b65b-65b7a8bdb5be',
 '183297d1-50af-4a51-8f1c-035c8815e9dd',
 '4a4a1dfe-d224-41e7-853a-fbe6d7b31c0d',
 'cb2e8abf-5219-47f8-80a4-fce5b404195c',
 'ce5fd4be-e0ff-409f-bb22-f4f7fa10d471',
 '202d70ec-3a6c-4636-9f55-08e6a55684dc',
 'da43bdfb-c230-4379-9e51-9ed55a69c459',
 'f617c6aa-1196-4775-be46-64d4c7023c29',
 '3a26b362-da08-482b-87b2-95aca6a40f4d',
 'c076fa1e-a367-43f8-8989-6837f43bbfe0',
 'aaf63c58-5b9e-433e-b789-3a73737b3023',
 'db3bbd8e-6712-4f95-886f-67225c4b099f']

#### Searching granular chunks

In [14]:
results = corwnall_granular_collection.similarity_search(query="Events or festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='Festivals 
 [ edit ] 
 
 These festivals tend to not be public holidays and not all are celebrated fully across the county. 
   
 AberFest 
 .   A Celtic cultural festival celebrating “All things” Cornish and Breton that takes place biennially (every two years) in Cornwall at Easter. The AberFest Festival alternates with the Breizh – Kernow Festival that is held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate years.       ( updated Jun 2023 ) 
 Golowan , sometimes also  Goluan  or  Gol-Jowan  is the Cornish word for the Midsummer celebrations, most popular in the Penwith area and in particular  Penzance  and  Newlyn . The celebrations are conducted from the 23rd of June (St John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve being the more popular in Cornish fishing communities. The celebrations are centred around the lighting of bonfires and fireworks and the performanc

### Splitting into coarse chunks with the RecursiveCharacterTextSplitter 

In [15]:
from langchain_community.document_transformers import Html2TextTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [16]:
html2text_transformer = Html2TextTransformer()

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=300
)

In [18]:
def split_docs_into_coarse_chunks(docs):
    text_docs = html2text_transformer.transform_documents(docs) #A 
    coarse_chunks = text_splitter.split_documents(text_docs)

    return coarse_chunks
#A transform HTML docs into clean text docs

In [19]:
coarse_chunks = split_docs_into_coarse_chunks(docs)

#### Ingesting coarse chunks

In [20]:
corwnall_coarse_collection.add_documents(documents=coarse_chunks)

['27232a75-4a42-4282-ab64-3344e44cf96f',
 '312f1d28-b5e7-4ee3-a22c-63b7815307eb',
 'bac9163e-eaf6-4922-aa1c-59be1cbeeed2',
 'eda329d7-5f92-4116-a9d6-687dc5dbb7f0',
 'dac09a60-99a2-47ea-90a0-0d1cdc21995f',
 '74fb0ed7-26b3-4bec-82ea-f1444e051e8a',
 '25466426-2621-4a9e-b0a6-43a6e2c5b74d',
 '041e677f-add7-4046-8d4f-6d5a62f17e11',
 '0a6512e8-85a9-44d3-a73b-1e4e77619dd2',
 '080564f9-0183-47a1-b96d-f99d1ea4fa09',
 '30968ebc-7ed6-4c73-a6d3-c37aaa8acbfd',
 'f1f37961-9224-4e52-a56b-cf2661eb6dd4',
 'be473a95-605a-425f-ad98-629fd8688b39',
 '42219dec-1af4-40c1-b716-8b44b9079922',
 '84cd07c8-d19e-4d8d-8860-7eb341fbb86b']

#### Searching coarse chunks

In [21]:
results = corwnall_coarse_collection.similarity_search(query="Events or festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='### Spirits

[edit]

    _See also:Liquor_

Gin and rum are also produced in Cornwall. A popular brand of Cornish rum is
Dead Man's Fingers which has multiple flavours and is bottled in St. Ives.

## Festivals

[edit]

These festivals tend to not be public holidays and not all are celebrated
fully across the county.

AberFest. A Celtic cultural festival celebrating “All things” Cornish and
Breton that takes place biennially (every two years) in Cornwall at Easter.
The AberFest Festival alternates with the Breizh – Kernow Festival that is
held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate
years. (updated Jun 2023)

**Golowan** , sometimes also _Goluan_ or _Gol-Jowan_ is the Cornish word for
the Midsummer celebrations, most popular in the Penwith area and in particular
Penzance and Newlyn. The celebrations are conducted from the 23rd of June (St
John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve
being the more popular in Corni

## Splitting and ingesting the content of various URLs (across UK destinations)

### Preparing the Chroma DB collections

In [22]:
uk_granular_collection = Chroma( #A
    collection_name="uk_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_granular_collection.reset_collection() #B

In [23]:
uk_coarse_collection = Chroma( #A
    collection_name="uk_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_coarse_collection.reset_collection() #B

### Splitting and ingesting HTML content with the HTMLSectionSplitter 

In [37]:
# Reduce this list if you want to save on processing fees
uk_destinations = [
    "Cornwall", "North_Cornwall", "South_Cornwall", "West_Cornwall", 
    "Tintagel", "Bodmin", "Wadebridge", "Penzance", "Newquay",
    "St_Ives", "Port_Isaac", "Looe", "Polperro", "Porthleven"
    "East_Sussex", "Brighton", "Battle", "Hastings_(England)", 
    "Rye_(England)", "Seaford", "Ashdown_Forest"
] 

wikivoyage_root_url = "https://en.wikivoyage.org/wiki"

In [25]:
uk_destination_urls = [f'{wikivoyage_root_url}/{d}' for d in uk_destinations]

In [26]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #C
    docs =  html_loader.load() #D
    
    for doc in docs:
        print(doc.metadata)
        granular_chunks = split_docs_into_granular_chunks(docs)
        uk_granular_collection.add_documents(documents=granular_chunks)

        coarse_chunks = split_docs_into_coarse_chunks(docs)
        uk_coarse_collection.add_documents(documents=coarse_chunks)
#A Create a Chroma DB collection
#B Reset the collection in case it already exists 
#C Loader for one destination
#D Documents of one destination 

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.99it/s]


{'source': 'https://en.wikivoyage.org/wiki/Cornwall', 'title': 'Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.73it/s]


{'source': 'https://en.wikivoyage.org/wiki/North_Cornwall', 'title': 'North Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  2.47it/s]


...

{'source': 'https://en.wikivoyage.org/wiki/Rye_(England)', 'title': 'Rye (England) – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.40it/s]


{'source': 'https://en.wikivoyage.org/wiki/Seaford', 'title': 'Seaford – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.30it/s]


{'source': 'https://en.wikivoyage.org/wiki/Ashdown_Forest', 'title': 'Ashdown Forest – Travel guide at Wikivoyage', 'language': 'en'}


#### Searching 

In [27]:
granular_results = uk_granular_collection.similarity_search(query="Events or festivals in East Sussex",k=4)
for doc in granular_results:
    print(doc)

page_content='Brighton' metadata={'Header 1': 'Brighton'}
page_content='Seaford' metadata={'Header 1': 'Seaford'}
page_content='Penzance' metadata={'Header 1': 'Penzance'}
page_content='Festivals 
 [ edit ] 
 
 These festivals tend to not be public holidays and not all are celebrated fully across the county. 
   
 AberFest 
 .   A Celtic cultural festival celebrating “All things” Cornish and Breton that takes place biennially (every two years) in Cornwall at Easter. The AberFest Festival alternates with the Breizh – Kernow Festival that is held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate years.       ( updated Jun 2023 ) 
 Golowan , sometimes also  Goluan  or  Gol-Jowan  is the Cornish word for the Midsummer celebrations, most popular in the Penwith area and in particular  Penzance  and  Newlyn . The celebrations are conducted from the 23rd of June (St John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve being the more popular in Cornish 

In [28]:
coarse_results = uk_coarse_collection.similarity_search(query="Events or festivals in East Sussex",k=4)
for doc in coarse_results:
    print(doc)

page_content='### Events

[edit]

A market during the Brighton Festival. The Theatre Royal is the red building.
A colourful parade down Queens Road during Pride in 2016.

  * **Brighton Racecourse** has flat-racing April-Oct. It's on Freshfield Rd a mile east of town centre.
  * **Plumpton Racecourse** is National Hunt (jumps races) Nov-March, but it's 10 mi (16 km) north in Lewes.
  * Brighton Festival Fringe: early May – early June, ☏ +44 1273 764900, info@brightonfringe.org. The Fringe runs at the same time as the main festival, and features over 600 events, including comedy, theatre, music, and "open houses" (local artists exhibiting in their own homes) and tours (haunted pubs, Regency Brighton, churches, cemeteries, sewers, etc.)_(date needs fixing)_
  * Brighton Festival: May, ☏ +44 1273 709709 (tickets), tickets@brightonfestival.org. The Brighton Festival, in May each year, is the second biggest arts festival in Great Britain (coming closely behind Edinburgh). Music of all sorts

In [28]:
granular_results = uk_granular_collection.similarity_search(query="Beaches in Conrwall",k=4)
for doc in granular_results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='North Cornwall' metadata={'Header 1': 'North Cornwall'}
page_content='West Cornwall' metadata={'Header 1': 'West Cornwall'}
page_content='South Cornwall' metadata={'Header 1': 'South Cornwall'}


In [29]:
coarse_results = uk_coarse_collection.similarity_search(query="Beaches in Cornwall",k=4)
for doc in coarse_results:
    print(doc)

page_content='**South Cornwall** is in Cornwall. It includes much of the stunning Cornish
coast along the English Channel of the Atlantic Ocean.

## Towns and villages

[edit]

Map of South Cornwall

  * 50.26-5.0511 Truro — Cornwall's main centre hosts the Royal Cornwall Museum
  * 50.3311-4.20212 Cawsand — overlooks Plymouth Sound; Cawsand is within Mount Edgcumbe Country Park
  * 50.15-5.073 Falmouth — famous for its beaches, it is home to the world's third largest natural harbour
  * 50.334-4.6334 Fowey — the Fowey Regatta in mid-August attracts many yachts and sailing boats
  * 50.354-4.4545 Looe — a summer resort place with a monkey sanctuary, and an active fishing village
  * 50.408-4.2126 Saltash — "Gateway to Cornwall", a small town on the Cornwall side of the Tamar crossings
  * 50.338-4.7957 St Austell — largest town in the county and home to the Eden Project, the world's largest greenhouse
  * 50.3314-4.75788 Charlestown — seaside town used as filming location for the TV sh

# Embedding strategy

## Embedding child chunks with ParentDocumentRetriever

In [30]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Setting up the Parent Document retriever

In [33]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_store = InMemoryStore() #E

parent_doc_retriever = ParentDocumentRetriever( #F
    vectorstore=child_chunks_collection,
    docstore=doc_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Splitter to generate child granular chunks from parent coarse chunks
#C Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [34]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    print(f'Ingesting {destination_url}')
    parent_doc_retriever.add_documents(text_docs, ids=None) #D

#A Loader for destination web page
#B HTML documents of one destination 
#C Transform HTML docs into clean text deocs
#D Ingest coarse chunks into document store and granular chunks into vector store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 11.27it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 15.88it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 17.27it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 14.83it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 14.11it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


In [35]:
list(doc_store.yield_keys())
#A Show the keys of the added coarse chunks

['4bc51b46-41db-41c4-b6bc-85dbcfad8fbc',
 '897ba8c3-2954-4853-a61b-77debfcb0417',
 '06938d31-1808-47b3-8d51-e787fdf5cb2c',
 '8e620218-1ff3-4c89-9ba4-befe16689a6c',
 '849097d3-380a-4736-a2ba-62b5e93ba0de',
 '746264d6-fe04-4137-b86e-394aeea9d720',
 '366f12d5-1b20-42a9-88d2-99a192f2104e',
 'f6f73640-42e3-4364-9997-3154845f592a',
 'ec7b597d-f90c-4437-a017-2264dc116efd',
 'b1fea0e2-99a9-4f42-b615-b4fa75dda6bc',
 '1c35cb8e-7364-4268-9217-c4c53329e641',
 '964fa280-6881-49a5-bb0f-f5761aab59b8',
 '26de82f2-cdfa-4442-af64-c43141c9bfb9',
 '0908c3d7-83c8-4e2f-82eb-84f92ac4bc43',
 '25adcbd5-d705-4dfe-9084-ad963ea0545b',
 '75efd3b4-51eb-4194-8589-90cbf587afa4',
 'c702a9fc-b46b-4804-8957-4698640ddb11',
 'b0c44e37-f805-4029-846a-6c5fa2974699',
 '8d2a1195-5833-43ca-bcf0-25ebb1fa95dc',
 'e69ff852-2bde-4977-8094-0fb17b7f851c',
 '0fe2d83f-aa26-4e49-b7c0-2ecc8336d3e7',
 '8c05b58d-f61d-467b-9af2-a1ae097c4ca6',
 'ebae8aa6-9cfa-4388-8449-19ebfbacf9ad',
 'fb64a38b-870e-47ef-ab86-e51cb0a6cd7a',
 '349b7f0f-ede5-

### Performing a search on granular information 

In [45]:
retrieved_docs = parent_doc_retriever.invoke("Cornwall Ranger")

In [46]:
len(retrieved_docs)

4

In [47]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All D

### Comparing with direct semantic search on child chunks

In [48]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [49]:
len(child_docs_only)

4

In [50]:
child_docs_only[0]

Document(metadata={'doc_id': '03d8e7e2-27d4-4921-8457-2487408a4c72', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [51]:
# IMPORTANT: as you can see a granular search would have identified the chunk, but it would have lost the usefulcontext about travelling in Cornwall

## Embedding child chunks with MultiVectorRetriever

In [52]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [53]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_byte_store = InMemoryByteStore() #E
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #F
    vectorstore=child_chunks_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Splitter to generate child granular chunks from parent coarse chunks
#C Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [54]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_granular_chunks = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        granular_chunks = child_splitter.split_documents([coarse_chunk]) #F

        for granular_chunk in granular_chunks:
            granular_chunk.metadata[doc_key] = coarse_chunk_id #G

        all_granular_chunks.extend(granular_chunks)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_granular_chunks) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into parent coarse chunks
#E Iterate over the parent coarse chunks
#F Create child granular chunks form each parent coarse chunk
#G Link each child granular chunk to its parent coarse chunk
#H Ingest the child granular chunks into the vector store
#I Ingest the parent coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.63it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  5.44it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.44it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.35it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.01it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [55]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall Ranger")

In [56]:
len(retrieved_docs)

4

In [57]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All D

In [36]:
##IMPORTANT: same as Parent Document retriever, but more control and flexibility on how to link child to parent chunks

### Comparing with direct semantic search on child chunks

In [59]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [60]:
len(child_docs_only)

4

In [61]:
child_docs_only[0]

Document(metadata={'doc_id': '631e7780-e23d-4ca9-83fb-b0be24091611', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [62]:
## IMPORTANT: Same as before

## Embedding summaries with MultiVectorRetriever

In [63]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid

### Setting up the Multi vector retriever (similar to when embedding child chunks)

In [64]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A

summaries_collection = Chroma( #B
    collection_name="uk_summaries",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

summaries_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=summaries_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the summarization chain

In [65]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

In [66]:
summarization_chain = (
    {"document": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{document}") #B
    | llm
    | StrOutputParser())

#A Grab the text content from the document
#B Instantiate a prompt asking to generate summary of the provided text
#C Send the LLM the instantiated prompt 
#D Extract the summary text from the response

### Ingesting the coarse chunks and related summaries into doc and vector store

In [67]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_summaries = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        summary_text =  summarization_chain.invoke(coarse_chunk) #F
        summary_doc = Document(page_content=summary_text, metadata={doc_key: coarse_chunk_id})

        all_summaries.append(summary_doc) #G

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_summaries) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a summary for the coarse chunk thorugh the summarization chain
#G Link each summary to its related coarse chunk
#H Ingest the summaries into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.83it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.04it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.53it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  2.94it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.85it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


In [70]:
# COMMENT: the code above is similar to when ingesting child chunks, but it is slower because of the summarization step
# which invokes the LLM.
# The processing can be speeded up by parallelizing the outer for loop on the destination urls.

### Performing a search on granular information

In [71]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall travel")

In [72]:
len(retrieved_docs)

4

In [73]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All 

### Comparing with direct semantic search on summaries

In [74]:
summary_docs_only =  summaries_collection.similarity_search("Cornwall Travel")

In [75]:
len(summary_docs_only)

4

In [76]:
summary_docs_only

[Document(metadata={'doc_id': '68673c9f-4ae9-4026-8f73-abd7218557f6'}, page_content="The document is a travel guide excerpt from Wikivoyage about Cornwall, a county in the southwest of the United Kingdom. It describes Cornwall as a distinct and popular tourist destination known for its warm climate, long coastline, stunning scenery, and rich Celtic heritage, including legends of pirates and King Arthur. The region is recognized for its cultural tourism and archaeological significance, with over 30% of the area designated as an Area of Outstanding Natural Beauty (AONB).\n\nThe guide outlines various sections, including regions, towns and cities, visitor information, transportation options, attractions like National Trust properties, local cuisine, beverages, festivals, accommodation, and safety tips. Cornwall is divided into three main regions: North Cornwall, South Cornwall, and West Cornwall, each offering unique landscapes and experiences. The document emphasizes the area's pride in 

In [77]:
# COMMENT: a direct search on summaries retrieves denser information, but it is missing out on useful details. 
# However, you might consider using the summaries directly if after testing they prove adequate.

## Embedding hypothetical questions with MultiVectorRetriever

In [57]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid
from typing import List
from pydantic import BaseModel, Field

### Setting up the Multi vector retriever (same as when embedding summaries)

In [59]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A

hypothetical_questions_collection = Chroma( #B
    collection_name="uk_hypothetical_questions",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

hypothetical_questions_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=hypothetical_questions_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the chain to generate hypothetical questions

In [60]:
class HypotheticalQuestions(BaseModel):
    """Generate hypothetical questions for given text."""

    questions: List[str] = Field(..., description="List of hypothetical questions for given text")

In [61]:
llm_with_structured_output = ChatOpenAI(model="gpt-4o-mini", 
        openai_api_key=OPENAI_API_KEY).with_structured_output(
        HypotheticalQuestions
)

In [62]:
hypothetical_questions_chain = (
    {"document_text": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template( #B
        "Generate a list of exactly 4 hypothetical questions that the below text could be used to answer:\n\n{document_text}"
    )
    | llm_with_structured_output #C
    | (lambda x: x.questions) #D
)

#A Grab the text content from the document
#B Instantiate a prompt asking to generate 4 hypothetical questions on the provided text
#C Invoke the LLM configured to return an object containing the questions as a typed list of strings
#D Grab the list of questions from the response

### Ingesting the coarse chunks and related hypothetical questions into doc and vector store

In [63]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_hypothetical_questions = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        hypothetical_questions = hypothetical_questions_chain.invoke(coarse_chunk) #F
        hypothetical_questions_docs = [Document(page_content=question, metadata={doc_key: coarse_chunk_id})
                                              for question in hypothetical_questions] #G

        all_hypothetical_questions.extend(hypothetical_questions_docs)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_hypothetical_questions) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a list of hypothetical questions for the coarse chunk thorugh the question generation chain
#G Link each hypothetical question to its related coarse chunk
#H Ingest the hypothetical questions into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 13.10it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 16.05it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 16.75it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 17.99it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 15.04it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [64]:
retrieved_docs = multi_vector_retriever.invoke("How can you go to Brighton from London?")

In [65]:
len(retrieved_docs)

3

In [66]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/Brighton', 'title': 'Brighton – Travel guide at Wikivoyage', 'language': 'en'}, page_content='## Go next\n\n[edit]\n\n  * Lewes – this gorgeous medieval town has a castle, as well as **Glyndebourne Opera House** , a famous opera house, set in beautiful grounds where opera-goers eat gourmet picnics at intervals which can be brought in or ordered from their own catering service. 11 mi (18 km) from Brighton and 20 minutes away by train.\n  * Dieppe, France – by ferry from Newhaven Harbour, about 9 mi (14 km) east of Brighton. Services are three daily and take around four hours. The service is operated by DFDS\n  * Rottingdean – in the east of Brighton, with memories of Kipling, Burne-Jones and several other artists.\n  * London – an hour away via train or 2 hours via coach.\n  * Worthing – There is no reason you wouldn\'t want to visit this lovely town. Short journey via train from Brighton Station.\n\n**Routes through Brighton

### Inspecting possible questions matching our question through semantic search

In [67]:
hypothetical_question_docs_only =  hypothetical_questions_collection.similarity_search("How can you go to Brighton from London?")

In [68]:
len(hypothetical_question_docs_only)

4

In [69]:
hypothetical_question_docs_only

[Document(id='91837b61-f3c7-4fe2-9c81-56e660b68998', metadata={'doc_id': '1c0a73eb-9f2d-414c-b7e7-878d4509f0bd'}, page_content='What if someone wanted to plan a day trip from London to Brighton, what transportation options could they consider?'),
 Document(id='d19d9895-1c16-4ef2-86b5-e231fc0b68f8', metadata={'doc_id': 'f16aea58-1ed5-4c7a-a0ef-676e64cd55e6'}, page_content='What would it be like to travel to Brighton from Gatwick Airport using the train?'),
 Document(id='1bd3353f-2afa-4cba-9790-aec9a1d546fa', metadata={'doc_id': 'd9c56504-d613-447a-8c00-56f5ac39d10d'}, page_content='How could someone navigate around Brighton using public transportation?'),
 Document(id='b6822b8b-3193-48bc-a2f3-f47c7d2e8628', metadata={'doc_id': 'd9c56504-d613-447a-8c00-56f5ac39d10d'}, page_content='What would a traveler need to know before visiting Brighton?')]

# Granular chunk expansion with MultiVectorRetriever

In [70]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [71]:
granular_chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #A

granular_chunks_collection = Chroma( #B
    collection_name="uk_granular_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

granular_chunks_collection.reset_collection() #C

expanded_chunk_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=granular_chunks_collection,
    byte_store=expanded_chunk_store
)
#A Splitter to generate granular chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host expanded chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Ingesting granular and expanded chunks into doc and vector store

In [72]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    granular_chunks = granular_chunk_splitter.split_documents(text_docs) #D

    expanded_chunk_store_items = []
    for i, granular_chunk in enumerate(granular_chunks): #E

        this_chunk_num = i #F
        previous_chunk_num = i-1 #F
        next_chunk_num = i+1 #F
        
        if i==0: #F
            previous_chunk_num = None
        elif i==(len(granular_chunks)-1): #F
            next_chunk_num = None

        expanded_chunk_text = "" #G
        if previous_chunk_num: #G
            expanded_chunk_text += granular_chunks[previous_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_text += granular_chunks[this_chunk_num].page_content #G
        expanded_chunk_text += "\n"

        if next_chunk_num: #G
            expanded_chunk_text += granular_chunks[next_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_id = str(uuid.uuid4()) #H
        expanded_chunk_doc = Document(page_content=expanded_chunk_text) #I

        expanded_chunk_store_item = (expanded_chunk_id, expanded_chunk_doc)
        expanded_chunk_store_items.append(expanded_chunk_store_item)

        granular_chunk.metadata[doc_key] = expanded_chunk_id #J
            
    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(granular_chunks) #K
    multi_vector_retriever.docstore.mset(expanded_chunk_store_items) #L

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into granular chunks
#E Iterate over the granular chunks
#F determine the index of the current chunk and its previous and next chunks
#G Assemble the text of the expanded chunk by including the previous and next chunk
#H Generate the ID of the expanded chunk
#I Create the expanded chunk document
#J Link each granular chunk to its related expanded chunk
#K Ingest the granular chunks into the vector store
#L Ingest the expanded chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 14.71it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 17.54it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 13.44it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 18.29it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 13.52it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [193]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall Ranger")

In [194]:
len(retrieved_docs)

4

In [195]:
retrieved_docs[0]

Document(page_content="Buses only serve designated stops when in towns; otherwise, you can flag them\ndown anywhere that's safe for them to stop.\n\n### By train\n\n[edit]\n\n**CrossCountry Trains** and **Great Western Railway** operate regular train\nservices between the main centres of population, the latter company also\nserving a number of other towns on branch lines. For train times and fares\nvisit National Rail Enquiries.\nThe **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n\nThe **Lost Gardens of

### Comparing with direct semantic search on granular chunks

In [196]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [197]:
len(child_docs_only)

4

In [198]:
child_docs_only[0]

Document(metadata={'doc_id': '04c7f88e-e090-4057-af5b-ea584e777b3f', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [ ]:
# COMMENT: the expanded chunk has more useful context